### **TRANSFORMERS**

In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install clip --quiet

In [4]:

from PIL import Image
from transformers import CLIPImageProcessor, CLIPModel

# Load the CLIP model and processor#openai/clip-vit-base-patch32
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").vision_model
clip_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load your image
image_path = "/content/sample_image.jpeg"  # Replace with the path to your image
image = Image.open(image_path)

# Preprocess the image
inputs = clip_processor(text=None, images=image, return_tensors="pt")



ModuleNotFoundError: No module named 'transformers'

In [ ]:
# Encode the image
with torch.no_grad():
    image_features = clip_model(**inputs).pooler_output


# The `image_features` now contains the encoded image as a tensor



In [ ]:
ind = torch.argmax(image_features,dim=-1)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [ ]:
# Initialize the GPT-2 tokenizer and model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')



In [ ]:
# Generate text based on the encoded tokens
input_ids = ind.unsqueeze(0)  # Add batch dimension


In [ ]:
with torch.no_grad():
    generated_output = gpt2_model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the output
generated_text = gpt2_tokenizer.decode(generated_output[0], skip_special_tokens=True)

print("Generated text:")
print(generated_text)


In [ ]:
gpt2_model(inputs_embeds=torch.randn((1,768))).logits.shape

In [ ]:
import gdown

MODEL_WEIGHTS = {
    "L": "1Gh32arzhW06C1ZJyzcJSSfdJDi3RgWoG",
    "S": "1pSQruQyg8KJq6VmzhMLFbT_VaHJMdlWF",
}


def download_model(checkpoint_fpath, model_size="S"):
    """
    Downloads weights from Google Drive.
    """

    download_id = MODEL_WEIGHTS[model_size.strip().upper()]

    gdown.download(
        f"https://drive.google.com/uc?id={download_id}", checkpoint_fpath, quiet=False
    )
download_model("/content/drive/MyDrive/Drivebit")

In [3]:
!export KAGGLE_USERNAME=iamtars
!export KAGGLE_KEY=e10e91505fe6ab8a093cc0f348efb35e
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d adityajn105/flickr8k
import subprocess
zip_file_path = "/content/flickr8k.zip"
extract_path = "/content/flickr8k"
subprocess.run(["unzip", zip_file_path, "-d", extract_path])

'export' is not recognized as an internal or external command,
operable program or batch file.


'export' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '~/.kaggle/kaggle.json': No such file or directory
'kaggle' is not recognized as an internal or external command,
operable program or batch file.


CompletedProcess(args=['unzip', '/content/flickr8k.zip', '-d', '/content/flickr8k'], returncode=9)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import os
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


class Dataset(Dataset):
    def __init__(self):
        self.imgspath = os.listdir("/content/flickr8k/Images")
        with open("/content/flickr8k/captions.txt", 'r') as file:
          lines = file.readlines()
        self.captions = {}
        lines = lines[1:]
        for line in lines:
          if ',' in line:
            image, caption = line.strip().split(",", 1)
            self.captions[image] = caption
    def __len__(self):
        return len(self.imgspath)

    def __getitem__(self, idx):
        img = os.path.join("/content/flickr8k/Images",self.imgspath[idx])
        label = self.captions[self.imgspath[idx]]
        img = Image.open(img)
        img = transform(img)
        return img, (label)
batch_size = 32
dataset = Dataset()
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to display a batch of images with their corresponding labels
def show_images(images, labels):
    num_images = len(images)
    rows = int(np.ceil(num_images / 2))  # Display 2 images per row
    fig, axs = plt.subplots(rows, 2, figsize=(20, 5 * rows))
    axs = axs.ravel()
    for i in range(rows * 2):
        if i < num_images:
            axs[i].imshow(np.transpose(images[i], (1, 2, 0)))  # Transpose to (H, W, C) for displaying with matplotlib
            axs[i].set_title(labels[i])
            axs[i].axis('off')
        else:
            axs[i].axis('off')  # Hide empty subplots
    plt.show()

# Iterate through the data loader and display a batch of images
for batch in data_loader:
    images, labels = batch
    show_images(images, labels)
    break  # Display only the first batch for simplicity


In [ ]:
len(data_loader)